# 16 - Decoradores: extender comportamiento sin tocar la funcion original

## Objetivos de aprendizaje

En esta sesion aprenderas a:

1. Entender que problema resuelven los decoradores.
2. Explicar la relacion entre funciones, closures y decoradores.
3. Construir decoradores basicos y robustos.
4. Usar `@decorador` entendiendo que hace Python por debajo.
5. Preservar metadatos con `functools.wraps`.
6. Crear decoradores con parametros (fabricas de decoradores).
7. Combinar decoradores y razonar sobre su orden de aplicacion.
8. Identificar errores sutiles (retornos perdidos, estado compartido, efectos colaterales).
9. Aplicar decoradores en casos reales: logging, validacion, cache y control de acceso.
10. Tomar decisiones de diseno: cuando conviene un decorador y cuando no.


## 1. Problema que resuelven los decoradores

A veces quieres agregar capacidades transversales a varias funciones:
- medir tiempo,
- registrar logs,
- validar argumentos,
- controlar permisos,
- cachear resultados.

Sin decoradores, repites codigo alrededor de la logica principal.
Con decoradores, encapsulas ese comportamiento extra en una sola pieza reutilizable.


In [ ]:
def saludar(nombre):
    print(f"Hola, {nombre}")

# Sin decorador: repeticion manual
print("[LOG] antes")
saludar("Ana")
print("[LOG] despues")


## 2. Base conceptual: funciones como objetos + closures

Un decorador es posible porque en Python:
1. Las funciones son objetos de primera clase.
2. Una funcion puede devolver otra funcion.
3. Las funciones internas pueden recordar variables del contexto externo (closure).

Sin esta base, decorar no tendria sentido.


In [ ]:
def envolver(contenido):
    prefijo = "[INFO]"

    def mensaje():
        return f"{prefijo} {contenido}"

    return mensaje

f = envolver("sistema listo")
print(f())


## 3. Primer decorador manual

Un decorador recibe una funcion y devuelve otra funcion.
La funcion retornada (wrapper) llama a la original y agrega comportamiento.


In [ ]:
def log_basico(func):
    def wrapper():
        print("[LOG] iniciando")
        resultado = func()
        print("[LOG] finalizado")
        return resultado

    return wrapper


def tarea():
    print("ejecutando tarea")


tarea_decorada = log_basico(tarea)
tarea_decorada()


## 4. Azucar sintactica: `@decorador`

Estas dos formas son equivalentes:

1. `f = decorador(f)`
2. usar `@decorador` justo encima de `def f(...):`

`@decorador` mejora legibilidad y deja clara la intencion.


In [ ]:
def log_basico(func):
    def wrapper():
        print("[LOG] iniciando")
        resultado = func()
        print("[LOG] finalizado")
        return resultado

    return wrapper


@log_basico
def proceso():
    print("proceso principal")


proceso()


## 5. Decoradores generales con `*args` y `**kwargs`

Error comun: crear wrappers que solo funcionan sin parametros.
Para ser reutilizable, el wrapper debe aceptar cualquier firma.


In [ ]:
def log_general(func):
    def wrapper(*args, **kwargs):
        print(f"[LOG] {func.__name__} args={args} kwargs={kwargs}")
        return func(*args, **kwargs)

    return wrapper


@log_general
def sumar(a, b):
    return a + b


print(sumar(3, 4))


## 6. Preservar metadatos con `functools.wraps`

Sin `@wraps`, la funcion decorada pierde datos importantes:
- `__name__`
- `__doc__`
- otras propiedades de introspeccion

Esto afecta debugging, ayuda interactiva y herramientas automaticas.


In [ ]:
from functools import wraps


def log_con_wraps(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"[LOG] entrando en {func.__name__}")
        return func(*args, **kwargs)

    return wrapper


@log_con_wraps
def area_rectangulo(base, altura):
    return base * altura


print(area_rectangulo.__name__)
print(area_rectangulo.__doc__)


## 7. Decoradores con parametros (fabrica de decoradores)

Cuando quieres configurar el decorador (por ejemplo nivel de log),
necesitas una funcion externa adicional.

Estructura tipica:
1. fabrica(parametros)
2. decorador(func)
3. wrapper(*args, **kwargs)


In [ ]:
from functools import wraps


def log_nivel(nivel):
    def decorador(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(f"[{nivel}] ejecutando {func.__name__}")
            return func(*args, **kwargs)

        return wrapper

    return decorador


@log_nivel("DEBUG")
def dividir(a, b):
    return a / b


print(dividir(10, 2))


## 8. Caso real 1: medicion de tiempo

Medir tiempos alrededor de funciones es un caso clasico.
`time.perf_counter()` es adecuado para medir intervalos cortos.


In [ ]:
from functools import wraps
from time import perf_counter, sleep


def medir_tiempo(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        inicio = perf_counter()
        resultado = func(*args, **kwargs)
        fin = perf_counter()
        print(f"{func.__name__} tomo {fin - inicio:.6f} s")
        return resultado

    return wrapper


@medir_tiempo
def trabajo_lento():
    sleep(0.05)
    return "ok"


print(trabajo_lento())


## 9. Caso real 2: validacion de argumentos

Un decorador puede verificar precondiciones antes de ejecutar logica principal.
Si la validacion falla, lanza excepcion con mensaje claro.


In [ ]:
from functools import wraps


def validar_no_vacio(func):
    @wraps(func)
    def wrapper(texto, *args, **kwargs):
        if not isinstance(texto, str) or not texto.strip():
            raise ValueError("texto invalido")
        return func(texto, *args, **kwargs)

    return wrapper


@validar_no_vacio
def normalizar(texto):
    return texto.strip().lower()


print(normalizar("  Hola  "))


## 10. Caso real 3: cache simple

Si una funcion pura recibe los mismos argumentos varias veces,
puedes guardar resultados para evitar recalculo.

Advertencia:
- Este ejemplo asume argumentos hashables.
- Para casos productivos, revisa `functools.lru_cache`.


In [ ]:
from functools import wraps


def cache_simple(func):
    memoria = {}

    @wraps(func)
    def wrapper(*args, **kwargs):
        clave = (args, tuple(sorted(kwargs.items())))
        if clave in memoria:
            print("cache hit")
            return memoria[clave]
        print("cache miss")
        valor = func(*args, **kwargs)
        memoria[clave] = valor
        return valor

    return wrapper


@cache_simple
def potencia(a, b):
    print("calculando...")
    return a ** b


print(potencia(2, 10))
print(potencia(2, 10))


## 11. Orden de aplicacion cuando apilas decoradores

Si tienes:

```python
@A
@B
def f():
    ...
```

Python interpreta: `f = A(B(f))`

Eso significa:
- `B` se aplica primero.
- luego `A` envuelve el resultado.

Razonar este orden evita bugs al combinar validacion, cache y logging.


In [ ]:
from functools import wraps


def deco_a(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("A antes")
        r = func(*args, **kwargs)
        print("A despues")
        return r

    return wrapper


def deco_b(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("B antes")
        r = func(*args, **kwargs)
        print("B despues")
        return r

    return wrapper


@deco_a
@deco_b
def accion():
    print("accion principal")


accion()


## 12. Decoradores y manejo de errores

Un decorador puede transformar excepciones o aplicar reintentos.
Pero debes evitar ocultar errores de forma silenciosa.

Regla util: si capturas una excepcion, decide explicitamente si:
- la vuelves a lanzar,
- devuelves un valor seguro,
- o la conviertes en otra mas semantica.


In [ ]:
from functools import wraps


def capturar_y_reportar(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except ZeroDivisionError as e:
            print(f"error en {func.__name__}: {e}")
            raise

    return wrapper


@capturar_y_reportar
def inverso(x):
    return 1 / x


print(inverso(2))
# print(inverso(0))


## 13. Decoradores basados en clase

No siempre un decorador debe ser funcion.
Una clase con `__call__` tambien puede actuar como decorador.

Ventaja: encapsular estado y comportamiento adicional de forma explicita.


In [ ]:
from functools import wraps


class ContadorLlamadas:
    def __init__(self, func):
        wraps(func)(self)
        self.func = func
        self.total = 0

    def __call__(self, *args, **kwargs):
        self.total += 1
        print(f"llamada #{self.total} -> {self.func.__name__}")
        return self.func(*args, **kwargs)


@ContadorLlamadas
def saludar(nombre):
    return f"Hola {nombre}"


print(saludar("Ana"))
print(saludar("Luis"))


## 14. Buenas practicas

1. Usa `@wraps` casi siempre.
2. Mantiene los decoradores pequenos y con una sola responsabilidad.
3. No ocultes errores importantes sin justificacion.
4. Documenta efectos secundarios (logs, cache, retries, permisos).
5. Evita decoradores "magicos" que dificulten leer el flujo.
6. Si la logica extra es muy especifica y local, una llamada explicita puede ser mejor que decorar.


## 15. Errores comunes

1. Olvidar `return func(...)` en el wrapper.
2. No aceptar `*args, **kwargs` y romper funciones con firma distinta.
3. Compartir estado mutable entre funciones sin querer.
4. Aplicar decoradores en orden incorrecto.
5. Usar cache sobre funciones con efectos secundarios.
6. Decorar demasiado y volver opaco el comportamiento real.


## 16. Ejercicios de pensamiento cuidadoso

Estos ejercicios apuntan a razonamiento tecnico, no solo sintaxis:
- composicion,
- flujo de ejecucion,
- diseno,
- y efectos secundarios.


### Ejercicio 1: refactor de repeticion transversal

**Tarea**: Tienes varias funciones con el mismo bloque de logging antes/despues.
Crea un decorador `loggear` y elimina repeticion sin cambiar el resultado final.

Objetivo: distinguir entre logica de negocio y logica transversal.


In [ ]:
# Tu codigo aqui
# def loggear(func):
#     ...


def crear_usuario(nombre):
    print("creando usuario", nombre)


def borrar_usuario(nombre):
    print("borrando usuario", nombre)

# Aplica el decorador en ambas funciones.


### Ejercicio 2: bug silencioso por retorno perdido

**Tarea**: Este decorador imprime mensajes, pero la funcion decorada siempre retorna `None`.
Encuentra el error y corrigelo.


In [ ]:
def auditoria(func):
    def wrapper(*args, **kwargs):
        print("inicio")
        func(*args, **kwargs)
        print("fin")
    return wrapper


@auditoria
def multiplicar(a, b):
    return a * b


print(multiplicar(3, 4))

# Tu correccion aqui


### Ejercicio 3: preservacion de metadatos

**Tarea**: Decora una funcion documentada y compara `__name__` y `__doc__`
antes y despues de usar `functools.wraps`.

Explica por que importa en proyectos reales.


In [ ]:
# Tu codigo aqui
# 1) crea un decorador sin wraps
# 2) aplica a una funcion con docstring
# 3) repite con wraps y compara


### Ejercicio 4: decorador parametrizable de permisos

**Tarea**: Implementa `requiere_rol(rol_minimo)`.
La funcion decorada recibira `usuario` (dict con campo `rol`).
Si no cumple el rol, debe lanzar `PermissionError`.

Piensa como representar jerarquia de roles de forma limpia.


In [ ]:
# Tu codigo aqui
# def requiere_rol(rol_minimo):
#     ...

# roles sugeridos: invitado < editor < admin


### Ejercicio 5: razonar orden de decoradores

**Tarea**: Predice la salida exacta sin ejecutar. Luego verifica.
Finalmente, invierte el orden y explica la diferencia.


In [ ]:
from functools import wraps


def A(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("A+")
        r = func(*args, **kwargs)
        print("A-")
        return r

    return wrapper


def B(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("B+")
        r = func(*args, **kwargs)
        print("B-")
        return r

    return wrapper


@A
@B
def f():
    print("X")


f()


### Ejercicio 6: reintentos con limites claros

**Tarea**: Implementa `reintentar(max_intentos, excepciones)`.
Debe reintentar solo para las excepciones indicadas y relanzar al agotar intentos.

Pregunta de diseno: conviene dormir entre intentos? quien define ese tiempo?


In [ ]:
# Tu codigo aqui
# def reintentar(max_intentos, excepciones=(Exception,)):
#     ...

# Prueba con una funcion que falla 2 veces y luego funciona.


### Ejercicio 7: cache y argumentos mutables

**Tarea**: Este enfoque falla con listas por no ser hashables.
Propone dos soluciones y analiza tradeoffs.

1. Convertir entradas mutables a inmutables.
2. Restringir API a argumentos hashables.


In [ ]:
# Caso problematico
# clave = (args, tuple(sorted(kwargs.items())))
# args podria contener listas o dicts.

# Tu propuesta aqui


### Ejercicio 8: rate limit testeable

**Tarea**: Crea `limitar_llamadas(max_llamadas, ventana_segundos, reloj)`.
`reloj` sera una funcion inyectada para facilitar pruebas.
Si se excede el limite, lanzar `RuntimeError`.

Clave del ejercicio: diseno para testabilidad.


In [ ]:
# Tu codigo aqui
# def limitar_llamadas(max_llamadas, ventana_segundos, reloj):
#     ...

# Pista: guarda timestamps recientes en un closure.


### Ejercicio 9: decorador de clase con estadisticas

**Tarea**: Implementa un decorador basado en clase `MetricasLlamada`
que registre:
1. total de llamadas,
2. ultima duracion,
3. promedio acumulado.

Debe funcionar con cualquier firma de funcion.


In [ ]:
# Tu codigo aqui
# class MetricasLlamada:
#     def __init__(self, func):
#         ...
#     def __call__(self, *args, **kwargs):
#         ...


### Ejercicio 10: criterio de arquitectura

**Tarea**: Para cada caso, decide si usar decorador o llamada explicita y justifica:
1. validar formato de email en 2 funciones.
2. abrir/cerrar transaccion en 30 operaciones de BD.
3. transformar un resultado solo en un endpoint unico.
4. registrar auditoria legal obligatoria en multiples acciones criticas.

No hay una unica respuesta correcta; evalua claridad, trazabilidad y mantenimiento.


In [ ]:
# Tu respuesta tecnica aqui


## 17. Resumen de conceptos clave

1. Un decorador envuelve una funcion para extender comportamiento.
2. `@decorador` es sintaxis para reasignar funciones.
3. `*args`, `**kwargs` y `@wraps` son piezas fundamentales.
4. El orden de decoradores cambia el comportamiento final.
5. Decorar bien mejora reutilizacion; decorar mal oculta logica.


## 18. Reto final opcional

Disena un mini sistema de tareas con tres decoradores combinables:
1. `@requiere_rol(...)`
2. `@medir_tiempo`
3. `@auditar(evento=...)`

Objetivo: razonar orden, errores, trazabilidad y legibilidad del flujo.
